# Purkinje-UV: Ellipsoid Demo

This minimal example walks through the full flow: **mesh → growth → Purkinje tree → activation → save → visualize**.  
By the end, you will have a VTU file with activation times and an interactive 3D visualization.

## What you’ll do
1. Define `FractalTreeParameters`
2. Grow the fractal tree on the surface.
3. Build a `PurkinjeTree` and run FIM activation.
4. Save a VTU file with activation times.
5. Load the VTU with `meshio` for a quick sanity check.
6. Visualize the result in 3D with `pyvista`.

## Environment
- Python ≥ 3.10.
- Tested packages (install all of them):
  - `purkinje_uv`
  - `numpy`
  - `meshio`
  - `pyvista`
  - `pyvistaqt` (Qt-based interactive backend for PyVista)

### Installation
Use one of the following:

**From PyPI (if available)**
```bash
pip install purkinje_uv numpy meshio pyvista pyvistaqt

In [ ]:
# If running on Colab, this installs everything you need.
# If you already have these installed locally, you can skip this cell.

%pip -q install --upgrade pip
%pip -q install purkinje_uv numpy meshio pyvista pyvistaqt pythreejs

In [1]:
from __future__ import annotations

import logging
from pathlib import Path

import numpy as np
import meshio
import pyvista as pv

from purkinje_uv import FractalTree, FractalTreeParameters, PurkinjeTree

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("purkinje_uv.examples.ellipsoid_demo")

print("Imports OK.")

Import of Cupy failed. The GPU version of fimpy will be unavailable. Message: No module named 'cupy'
Imports OK.


## Download or reuse the `purkinje-uv` repository

We fetch the repository so we can reference `data/ellipsoid.obj` with a known relative path.  
The cell will try `git clone` first, then fall back to a ZIP download if needed.


In [2]:
from pathlib import Path

REPO_URL = "https://github.com/ricardogr07/purkinje-uv"
DEST_DIR = Path("external/purkinje-uv")

if not DEST_DIR.exists():
    DEST_DIR.parent.mkdir(parents=True, exist_ok=True)
    print("Cloning purkinje-uv…")
    !git clone --depth 1 {REPO_URL} {str(DEST_DIR)}
else:
    print("Using existing checkout:", DEST_DIR)

REPO_ROOT = DEST_DIR.resolve()
MESH_PATH = REPO_ROOT / "data" / "ellipsoid.obj"
assert MESH_PATH.exists(), f"Expected mesh not found at: {MESH_PATH}"

print("Repo root:", REPO_ROOT)
print("Mesh path:", MESH_PATH)

Cloning purkinje-uv…
Repo root: C:\git\external\purkinje-uv
Mesh path: C:\git\external\purkinje-uv\data\ellipsoid.obj


Cloning into 'external\purkinje-uv'...


## Locate mesh file
The examples assume an ellipsoid mesh at `data/ellipsoid.obj`.

In [4]:
meshfile = Path(MESH_PATH)
print(f"Using mesh: {MESH_PATH}")

Using mesh: C:\git\external\purkinje-uv\data\ellipsoid.obj


## Configure parameters

`FractalTree` reads `meshfile` from the parameters and computes UV scaling internally.  
We serialize parameters to JSON for reproducibility.

In [6]:
lseg = 0.01
p = FractalTreeParameters(
    meshfile=str(MESH_PATH),
    init_node_id=738,
    second_node_id=210,
    l_segment=lseg,
    init_length=0.3,
    length=0.15,
    fascicles_length=[20 * lseg, 40 * lseg],
    fascicles_angles=[-0.4, 0.5],  # radians
)

params_json = REPO_ROOT / "examples" / "params.json"
params_json.parent.mkdir(parents=True, exist_ok=True)
p.to_json_file(str(params_json))

print("Saved params to:", params_json)

INFO:purkinje_uv.fractal_tree_parameters:Initialized FractalTreeParameters: meshfile='C:\\git\\external\\purkinje-uv\\data\\ellipsoid.obj', init_node_id=738, second_node_id=210, N_it=10, init_length=0.3, length=0.15, branch_angle=0.15 rad, w=0.1, l_segment=0.01, fascicles=(2 items)


Saved params to: C:\git\external\purkinje-uv\examples\params.json


## Grow fractal tree on the surface

In [7]:
tree = FractalTree(p)
tree.grow_tree()

print(
    f"Tree grown → nodes={len(tree.nodes_xyz)}, "
    f"segments={len(tree.connectivity)}, end_nodes={len(tree.end_nodes)}"
)

INFO:purkinje_uv.mesh:Loaded OBJ from C:\git\external\purkinje-uv\data\ellipsoid.obj with 781 vertices and 1470 triangles
INFO:purkinje_uv.mesh:Mesh initialized with 781 vertices and 1470 triangles


computing uv map


INFO:purkinje_uv.mesh:Mesh initialized with 781 vertices and 1470 triangles


generation 0
generation 1
generation 2
generation 3
generation 4
generation 5
generation 6
generation 7
generation 8
generation 9
Tree grown → nodes=3250, segments=3249, end_nodes=143


## Build Purkinje tree, run activation (FIM), save VTU, and sanity-check with `meshio`

In [8]:
Ptree = PurkinjeTree(
    np.asarray(tree.nodes_xyz),
    np.asarray(tree.connectivity),
    np.asarray(tree.end_nodes),
)

act = Ptree.activate_fim([0], [0.0], return_only_pmj=False)
pmj = Ptree.pmj

assert isinstance(act, np.ndarray) and act.ndim == 1
assert pmj is not None and np.all((pmj >= 0) & (pmj < act.shape[0]))
print(f"Activation computed → length={act.shape[0]}, PMJs={pmj.size}")

out_dir = REPO_ROOT / "examples" / "output"
out_dir.mkdir(parents=True, exist_ok=True)
out_file = out_dir / "ellipsoid_purkinje_AT.vtu"
Ptree.save(str(out_file))
print("Saved:", out_file)

m = meshio.read(str(out_file))
print(
    f"meshio OK → points={m.points.shape[0]}, cells={sum(len(c.data) for c in m.cells)}"
)

INFO:purkinje_uv.purkinje_tree:PurkinjeTree initialized with 3250 nodes and 3249 edges
INFO:purkinje_uv.purkinje_tree:Activating Purkinje tree with FIM solver
INFO:purkinje_uv.purkinje_tree:Saving PurkinjeTree to VTK at C:\git\external\purkinje-uv\examples\output\ellipsoid_purkinje_AT.vtu


Activation computed → length=3250, PMJs=143
Saved: C:\git\external\purkinje-uv\examples\output\ellipsoid_purkinje_AT.vtu
meshio OK → points=3250, cells=3249


## Visualize with PyVista (inline)

We attempt to use the `pythreejs` backend for inline rendering.  
If the backend is unavailable, we’ll save a screenshot instead.

In [ ]:
def pick_scalar(ds):
    preferred = ["Activation", "AT", "activation_time", "time_activation"]
    for name in preferred:
        if name in ds.array_names:
            return name
    if ds.point_data:
        return list(ds.point_data.keys())[0]
    if ds.cell_data:
        return list(ds.cell_data.keys())[0]
    return None

def to_tubes(ds, radius=0.003):
    try:
        return ds.tube(radius=radius)
    except Exception:
        return ds

try:
    try:
        pv.set_jupyter_backend("pythreejs")
        print("Using PyVista backend: pythreejs")
    except Exception:
        print("pythreejs backend not available; will try screenshot fallback.")

    ds = pv.read(str(out_file))
    scal_name = pick_scalar(ds)
    vis = to_tubes(ds, radius=0.003)

    p = pv.Plotter()
    p.add_mesh(
        vis,
        scalars=scal_name if scal_name else None,
        show_scalar_bar=bool(scal_name),
        scalar_bar_args={"title": scal_name or ""},
    )
    try:
        p.show(title="Purkinje VTU (ellipsoid)")
    except Exception as e:
        print("Inline render failed, saving screenshot…", e)
        img_path = out_dir / "ellipsoid_render.png"
        p.show(screenshot=str(img_path))
        from IPython.display import Image, display
        display(Image(filename=str(img_path)))
except Exception as e:
    print("PyVista visualization failed:", e)

pythreejs backend not available; will try screenshot fallback.


INFO:trame_server.utils.namespace:Translator(prefix=None)
INFO:wslink.backends.aiohttp:awaiting runner setup
INFO:wslink.backends.aiohttp:awaiting site startup
INFO:wslink.backends.aiohttp:Print WSLINK_READY_MSG
INFO:wslink.backends.aiohttp:Schedule auto shutdown with timout 0
INFO:wslink.backends.aiohttp:awaiting running future
INFO:trame_server.controller:trigger(trigger__1)
INFO:trame_server.controller:trigger(trigger__2)
INFO:trame_server.controller:trigger(P_0x16fc89be350_0Camera)
INFO:trame_server.controller:trigger(P_0x16fc89be350_0AnimateStart)
INFO:trame_server.controller:trigger(P_0x16fc89be350_0AnimateStop)
INFO:trame_client.widgets.core:js_key = class
INFO:trame_client.widgets.core:js_key = style
INFO:trame_client.widgets.core:js_key = fluid
INFO:trame_client.widgets.core:js_key = class
INFO:trame_client.widgets.core:before: class = { 'rounded-circle': !P_0x16fc89be350_0_show_ui }
INFO:trame_server.utils.namespace:(prefix=None) token {
INFO:trame_server.state:has({ => {) = 

Widget(value='<iframe src="http://localhost:53710/index.html?ui=P_0x16fc89be350_0&reconnect=auto" class="pyvis…

INFO:aiohttp.access:127.0.0.1 [15/Aug/2025:18:01:54 -0600] "GET /index.html?ui=P_0x16fc89be350_0&reconnect=auto HTTP/1.1" 200 238 "-" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Code/1.103.1 Chrome/138.0.7204.100 Electron/37.2.3 Safari/537.36"
INFO:aiohttp.access:127.0.0.1 [15/Aug/2025:18:01:54 -0600] "GET /vue.global.js HTTP/1.1" 200 255 "http://localhost:53710/index.html?ui=P_0x16fc89be350_0&reconnect=auto" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Code/1.103.1 Chrome/138.0.7204.100 Electron/37.2.3 Safari/537.36"
INFO:aiohttp.access:127.0.0.1 [15/Aug/2025:18:01:54 -0600] "GET /assets/index-e80c1ba5.css HTTP/1.1" 200 237 "http://localhost:53710/index.html?ui=P_0x16fc89be350_0&reconnect=auto" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Code/1.103.1 Chrome/138.0.7204.100 Electron/37.2.3 Safari/537.36"
INFO:aiohttp.access:127.0.0.1 [15/Aug/2025:18:01:54 -0600] "GET /assets/